<a href="https://colab.research.google.com/github/SonnetSaif/object-detection_YOLOv8/blob/main/custom_object_detection_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics datasets

In [ ]:
import os
from tqdm import tqdm
from datasets import load_dataset

def dump_img_labels(data, split):
    data = data[split]
    for i, example in tqdm(enumerate(data)):
        image = example["image"]
        labels = example["litter"]["label"]
        bboxes = example["litter"]["bbox"]
        targets = []

        for label, box in zip(labels, bboxes):
            targets.append(f"{label} {box[0]} {box[1]} {box[2]} {box[3]}")
        with open(f"datasets/labels/{split}/{i}.txt", "w") as f:
            for target in targets:
                f.write(target + "\n")
        image.save(f"datasets/images/{split}/{i}.png")


if __name__== '__main__':
    dataset = load_dataset("kili-technology/plastic_in_river")
    # print(dataset)
    # print(dataset["train"][0])

    os.makedirs("datasets/images/train",exist_ok=True)
    os.makedirs("datasets/images/validation",exist_ok=True)
    os.makedirs("datasets/labels/train",exist_ok=True)
    os.makedirs("datasets/labels/validation",exist_ok=True)

    dump_img_labels(dataset, "train")
    dump_img_labels(dataset, "validation")

Generating train split:   0%|          | 0/3407 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/425 [00:00<?, ? examples/s]

In [6]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")

# Use the model
model.train(data="/content/custom.yaml",
            epochs=20,
            imgsz=(1280, 720),
            batch=4,
            optimizer="Adam",
            lr0=1e-3,)

Ultralytics YOLOv8.0.199 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/custom.yaml, epochs=20, patience=50, batch=4, imgsz=(1280, 720), save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, opt

KeyboardInterrupt: ignored

In [10]:
from ultralytics import YOLO
from PIL import Image
from datasets import load_dataset


dataset = load_dataset("kili-technology/plastic_in_river")
img = dataset["test"][12]["image"]

model = YOLO("runs/detect/train/weights/best.pt")
res = model.predict(img)[0]

print(res)
print(res.boxes)
res = res.plot(line_width=1)
res = res[:, :, ::-1]
res = Image.fromarray(res)
res.save("output.png")


0: 736x1280 1 PLASTIC_BOTTLE, 74.0ms
Speed: 3.3ms preprocess, 74.0ms inference, 1.8ms postprocess per image at shape (1, 3, 736, 1280)


ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'PLASTIC_BAG', 1: 'PLASTIC_BOTTLE', 2: 'OTHER_PLASTIC_WASTE', 3: 'NOT_PLASTIC_WASTE'}
orig_img: array([[[255, 254, 250],
        [255, 254, 250],
        [255, 254, 250],
        ...,
        [123, 142, 139],
        [118, 137, 134],
        [121, 142, 139]],

       [[255, 254, 250],
        [255, 254, 250],
        [255, 254, 250],
        ...,
        [120, 139, 136],
        [117, 136, 133],
        [120, 141, 138]],

       [[255, 254, 250],
        [255, 254, 250],
        [255, 254, 250],
        ...,
        [121, 140, 137],
        [121, 140, 137],
        [118, 139, 136]],

       ...,

       [[172, 170, 159],
        [172, 170, 159],
        [172, 170, 159],
        ...,
        [ 43,  76,  62],
        [ 43,  76,  62],
        [ 47,  80,  66]],

       [[171, 169, 158],
        [171, 169, 158],
        [171, 169, 158],
        ..